# Pretrained transformers

In the last notebook we explored using a LSTM network. The training score improved over the MLP approach but the same can not be said for the test score. In this attempt we will use a transformer network to try to obtain a better test scores.



In [ ]:
# If running on Google Colab, upload the supporting files to google drive
# then run this to import them. Also install simpletransformers.
try:
  from google.colab import drive
  drive.mount('/content/drive')
  !cp 'drive/My Drive/Colab/__init__.py' .
  !cp 'drive/My Drive/Colab/config.py' .
  !cp 'drive/My Drive/Colab/helper.py' .
  !cp 'drive/My Drive/Colab/train.pickle' .
  !pip install simpletransformers
except:
  pass

In [2]:
import wandb
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs)
import pandas as pd
import logging
from sklearn import metrics

from helper import load_data

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [3]:
x_train, x_validation, x_test, y_train, y_validation, y_test = load_data()

In [4]:
train_df = pd.DataFrame(dict(text=x_train, labels=y_train.values.tolist()))
validation_df = pd.DataFrame(dict(text=x_validation, labels=y_validation.values.tolist()))
train_df.head()

,text,labels
3166,"""\n Commons Admin Noticeboard beginning 3/22 P...","[0, 0, 0, 0, 0, 0]"
4118,"""\n\nI see. But, all the same, I haven't seen ...","[0, 0, 0, 0, 0, 0]"
5199,Yes?? I've just not been on msn recently as iv...,"[0, 0, 0, 0, 0, 0]"
4333,", oh shit, stop making warnings, motherfucker,...","[1, 1, 1, 0, 1, 0]"
564,That's in the version that TRP demand son havi...,"[0, 0, 0, 0, 0, 0]"


## Hyperparameter optimization

In [5]:
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'eval_loss', 'goal': 'minimize'},
    'parameters': {
        'num_train_epochs': {'min': 2, 'max': 4},
        'learning_rate': {'min': 4e-5, 'max': 7e-5},
        'train_batch_size': {'values': [8, 16, 32]},
    }
}
# sweep_id = wandb.sweep(sweep_config, project='Toxic comments')

In [6]:
model_args = MultiLabelClassificationArgs(
    overwrite_output_dir=True,
    no_save=True,
    max_seq_length=96,
    use_multiprocessing=True,
    evaluate_during_training=True,
    manual_seed=42,
    use_cached_eval_features=True,
    wandb_project='Toxic comments')

def train():
    wandb.init()
    model = MultiLabelClassificationModel(
        'xlmroberta',
        'xlm-roberta-base',
        num_labels=6,
        args=model_args,
        sweep_config=wandb.config)
    model.train_model(
        train_df,
        eval_df=validation_df,
        acc=metrics.roc_auc_score)
    model.eval_model(validation_df, acc=metrics.roc_auc_score)

    wandb.join()

# Optimize hyperparameters
# wandb.agent(sweep_id, train)

## Train final model using hyperparameters from wandb

In [ ]:
final_model_args = MultiLabelClassificationArgs(
    learning_rate=5.3e-5,
    num_train_epochs=5,
    train_batch_size=32,
    max_seq_length=256,
    overwrite_output_dir=True,
    use_multiprocessing=True,
    evaluate_during_training=True,
    manual_seed=42,
    use_cached_eval_features=True)

model = MultiLabelClassificationModel(
    'xlmroberta',
    'xlm-roberta-base',
    num_labels=6,
    args=final_model_args)
model.train_model(
    train_df,
    eval_df=validation_df,
    acc=metrics.roc_auc_score)

## Model evaluation on test set

In [9]:
test_df = pd.DataFrame(dict(text=x_test, labels=y_test.values.tolist()))
result, _, _ = model.eval_model(test_df, acc=metrics.roc_auc_score)
result

{'LRAP': 0.9879855555555557,
 'acc': 0.9797946485975007,
 'eval_loss': 0.13224998372793198}